# Classfication sample code
The goal is checking an input sentence is from sports news or financial news articles

# Prepare data

In [11]:
import csv
import numpy as np
import MeCab
from sklearn.model_selection import train_test_split
from gensim.models.keyedvectors import KeyedVectors
from keras.layers import Dense,LSTM,Activation,Dropout
from keras.models import Sequential

MAX_LEN = 45
MECAB = MeCab.Tagger("-Owakati")
SIZE_OF_W2V = 200
W2V = KeyedVectors.load_word2vec_format('../data/embeddings/entity_vector.model.bin', binary=True)

def prepare_y(y):
    if y == "fin":
        return [1,0]
    else:
        return [0,1]
    
def get_embeddings(token):
    try:
        return W2V[token]
    except:
        return [0] * SIZE_OF_W2V

def prepare_x(x):
    tmp = []
    tokens = MECAB.parse(x).split(" ")
    print(tokens)
    print(len(tokens))
    for token in tokens:
        if token is not "\n":
            tmp.append(get_embeddings(token))
    
    # zero padding    
    for i in range(0, MAX_LEN - len(tokens) + 1):
        tmp.append([0] * SIZE_OF_W2V)
    
    return tmp
    
x_data = []
y_data = []

with open('../data/training/news.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        x_data.append(prepare_x(row["x"]))
        y_data.append(prepare_y(row["y"]))
        
x_data = np.array(x_data)
y_data = np.array(y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

print(x_train.shape)
print(y_train.shape)

['東海大', '・', '海野', '隆司', '捕手', 'は', '左', '越え', '本塁打', 'を', '放っ', 'た', '。', '\n']
14
['消費', '税', 'の', '税率', 'が', '１０月', '１', '日', 'に', '８', '％', 'から', '１', '０', '％', 'へ', '引き上げ', 'られる', '。', '\n']
20
['春季', 'リーグ', '戦', 'の', 'MVP', 'は', '元気', 'いっぱい', 'だ', '。', '九', '産', '大', 'の', '武', '上', 'が', '4', '打数', '3', '安打', '3', '打点', 'と', '気', 'を', '吐い', 'た', '。', '\n']
30
['中国', '側', 'も', '同', '時刻', 'に', '報復', 'として', '、', '米国', 'から', 'の', '輸入', '品', '７', '５', '０', '億', 'ドル', '分', 'の', '３', '３', '％', 'を', '占める', '１', '７', '１', '７', '品目', 'に', '５', '％', 'の', '追加', '関税', 'を', 'かけ', 'た', '。', '\n']
42
['福岡', '六', '大学', '野球', '秋季', 'リーグ', '戦', '（', '西日本新聞社', '後援', '）', 'は', '31', '日', '、', '福岡', '市', 'の', '福', '工大', '野球', '場', 'で', '開幕', '戦', '3', '試合', 'が', '行わ', 'れ', 'た', '。', '\n']
33
['春季', 'リーグ', '戦', 'は', 'チーム', '最多', 'の', '4', '勝', 'を', '挙げ', 'ながら', '、', '最長', 'イニング', 'は', '6', '回', '1', '/', '3', '。', '\n']
23
['世界', '経済', 'の', '減速', 'が', '意識', 'さ', 'れる', 'なか', '、', '米', '中', 'の', '制裁', '関税

# Train model

In [12]:
def create_model():
    model = Sequential()
    # units means output units   
    model.add(LSTM(units=64, input_shape=(MAX_LEN, SIZE_OF_W2V)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

model = create_model()
model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)

Epoch 1/10
16/16 [==============================] - 2s 103ms/step - loss: 0.6929 - acc: 0.4688
Epoch 2/10
16/16 [==============================] - 1s 38ms/step - loss: 0.6891 - acc: 0.5938
Epoch 3/10
16/16 [==============================] - 1s 44ms/step - loss: 0.6662 - acc: 0.6562
Epoch 4/10
16/16 [==============================] - 1s 42ms/step - loss: 0.6045 - acc: 0.6875
Epoch 5/10
16/16 [==============================] - 1s 37ms/step - loss: 0.3806 - acc: 1.0000
Epoch 6/10
16/16 [==============================] - 1s 37ms/step - loss: 0.2045 - acc: 1.0000
Epoch 7/10
16/16 [==============================] - 1s 43ms/step - loss: 0.0843 - acc: 1.0000
Epoch 8/10
16/16 [==============================] - 1s 40ms/step - loss: 0.0303 - acc: 1.0000
Epoch 9/10
16/16 [==============================] - 1s 41ms/step - loss: 0.0199 - acc: 1.0000
Epoch 10/10
16/16 [==============================] - 1s 39ms/step - loss: 0.0088 - acc: 1.0000


# Test

In [13]:
results = model.evaluate(x_test, y_test)
print(f"error: {results[0]}, accuracy: {results[1]}")

5/5 [==============================] - 0s 37ms/step
error: 0.0062434314750134945, accuracy: 1.0


# Predict

In [18]:
def translate_y(y):
    print(y)
    if y.flat[0] >= y.flat[1]:
        return "financial"
    else:
        return "sports"
        
def do_prediction(x):
    label = translate_y(model.predict(np.array([prepare_x(x)])))
    print(f"{x} is {label} \n")

do_prediction("イラン核合意に締結したフランスのマクロン大統領は、アメリカとイランの関係改善に努めてきた。")
do_prediction("５回２死一、二塁から左前安打で出塁。")
do_prediction("１点リードの五回。先頭で打席に立ち、３球目のファウルが左足を直撃した。")
do_prediction("１０月１日の消費税１０％引き上げまで１カ月。")

['イラン', '核', '合意', 'に', '締結', 'し', 'た', 'フランス', 'の', 'マクロン', '大統領', 'は', '、', 'アメリカ', 'と', 'イラン', 'の', '関係', '改善', 'に', '努め', 'て', 'き', 'た', '。', '\n']
26
[[0.99629265 0.00341535]]
イラン核合意に締結したフランスのマクロン大統領は、アメリカとイランの関係改善に努めてきた。 is financial 

['５', '回', '２', '死', '一', '、', '二塁', 'から', '左', '前', '安打', 'で', '出塁', '。', '\n']
15
[[0.0085403  0.99602604]]
５回２死一、二塁から左前安打で出塁。 is sports 

['１', '点', 'リード', 'の', '五', '回', '。', '先頭', 'で', '打席', 'に', '立ち', '、', '３', '球', '目', 'の', 'ファウル', 'が', '左足', 'を', '直撃', 'し', 'た', '。', '\n']
26
[[0.00853086 0.9960551 ]]
１点リードの五回。先頭で打席に立ち、３球目のファウルが左足を直撃した。 is sports 

['１０月', '１', '日', 'の', '消費', '税', '１', '０', '％', '引き上げ', 'まで', '１', 'カ月', '。', '\n']
15
[[0.996292   0.00342015]]
１０月１日の消費税１０％引き上げまで１カ月。 is financial 

